# Logistic Regression for Sentiment Analysis

Adapted from http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/outofcore_modelpersistence.ipynb

<br>
<br>

## The IMDb Movie Review Dataset

In this section, we will train a simple logistic regression model to classify movie reviews from the 50k IMDb review dataset that has been collected by Maas et. al.

> AL Maas, RE Daly, PT Pham, D Huang, AY Ng, and C Potts. Learning word vectors for sentiment analysis. In Proceedings of the 49th Annual Meeting of the Association for Computational Lin- guistics: Human Language Technologies, pages 142–150, Portland, Oregon, USA, June 2011. Association for Computational Linguistics

[Source: http://ai.stanford.edu/~amaas/data/sentiment/]

The dataset consists of 50,000 movie reviews from the original "train" and "test" subdirectories. The class labels are binary (1=positive and 0=negative) and contain 25,000 positive and 25,000 negative movie reviews, respectively.
For simplicity, I assembled the reviews in a single CSV file.


In [1]:
import pandas as pd
# if you want to download the original file:
#df = pd.read_csv('https://raw.githubusercontent.com/rasbt/pattern_classification/master/data/50k_imdb_movie_reviews.csv')
# otherwise load local file
df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,This film was reeeeeeallyyyy bad! Was it meant...,0
49996,Meek and mild Edward G. Robinson (as Wilbert W...,0
49997,"I guess I have to write something here, althou...",1
49998,A while back I bought the Chinese box set of F...,1
49999,"When watching little man , you'll spend its ru...",0


Let us shuffle the class labels.

In [2]:
import numpy as np
## uncomment these lines if you have dowloaded the original file:
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df[['review', 'sentiment']].to_csv('shuffled_movie_data.csv', index=False)

<br>
<br>

## Preprocessing Text Data

Now, let us define a simple `tokenizer` that splits the text into individual word tokens. Furthermore, we will use some simple regular expression to remove HTML markup and all non-letter characters but "emoticons," convert the text to lower case, remove stopwords, and apply the Porter stemming algorithm to convert the words into their root form.

In [3]:
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords
from nltk.corpus import opinion_lexicon
from nltk.corpus import brown

stop = stopwords.words('english')
#sentiwords = opinion_lexicon.positive()
#sentiwords += opinion_lexicon.negative()
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    #text = [w for w in text.split() if w in sentiwords]
    tokenized = [porter.stem(w) for w in text]
    return tokenized

Let's give it at try:

In [4]:
tokenizer('This :) fair good happy is a <a> test! :-) </br>')

['fair', 'good', 'happi', 'test', ':)', ':)']

## Learning (SciKit)

First, we define a generator that returns the document body and the corresponding class label:

In [5]:
def stream_docs(path):
    with open(path, 'r') as csv:
        next(csv) # skip header
        for line in csv:
            text, label = line[:-3], int(line[-2])
            yield text, label

To conform that the `stream_docs` function fetches the documents as intended, let us execute the following code snippet before we implement the `get_minibatch` function:

In [6]:
next(stream_docs(path='shuffled_movie_data.csv'))

('"Not worth the video rental or the time or the occasional efforts.<br /><br />*Makeup that a child can do. *Acting was over done...poor directing. *Editing was very choppy...many things made no sense or just seemed gratuitous. *Sound was badly dubbed. *Music was highly inappropriate. *Casting was extremely off...must have been on crack. *Zombies that talk let alone...drive, dance, work...just pisses me off. *And the bad guy...Holy Crap! As horribly casted as he was...he was the best looking zombie of all. Which doesn\'t say much.<br /><br />The Cover Art was good but very deceiving...as was the Main Menu of the DVD...great artwork and music.<br /><br />DON""T BOTHER!"',
 0)

After we confirmed that our `stream_docs` functions works, we will now implement a `get_minibatch` function to fetch a specified number (`size`) of documents:

In [7]:
def get_minibatch(doc_stream, size):
    docs, y = [], []
    for _ in range(size):
        text, label = next(doc_stream)
        docs.append(text)
        y.append(label)
    return docs, y

Next, we will make use of the "hashing trick" through scikit-learns [HashingVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.HashingVectorizer.html) to create a bag-of-words model of our documents. Details of the bag-of-words model for document classification can be found at  [Naive Bayes and Text Classification I - Introduction and Theory](http://arxiv.org/abs/1410.5329).

In [20]:
from sklearn.feature_extraction.text import HashingVectorizer
vect = HashingVectorizer(decode_error='ignore', 
                         n_features=2**21,
                         preprocessor=None, 
                         ngram_range = (1, 1),
                         tokenizer=tokenizer)

# Excercise 1: define new features according this https://web.stanford.edu/~jurafsky/slp3/7.pdf
doc_stream = stream_docs(path='shuffled_movie_data.csv')
X_in, y_in = get_minibatch(doc_stream, size=45000)
X_test, y_test = get_minibatch(doc_stream, size=5000)
X_in = vect.transform(X_in) 
X_test = vect.transform(X_test)

Using the [SGDClassifier]() from scikit-learn, we will can instanciate a logistic regression classifier that learns from the documents incrementally using stochastic gradient descent. 

In [21]:
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
import math
import pyprind
pbar = pyprind.ProgBar(2700)

def exp_fun(a):
    return 1/(1+math.exp(-a))

# Excercise 2: implement a MaxEnt classifier, using regularization, according this https://web.stanford.edu/~jurafsky/slp3/7.pdf
W = np.random.rand(1,2097152)
b = np.random.rand(1,1)
alpha = 0.01
regul = 0
error = 1

Wa = 0 # acumulado
ba = 0 # acumulado

for k in range(60):
    for i in range(45):
        X_train = X_in[1000*i:1000*(i+1)]
        y_train = y_in[1000*i:1000*(i+1)]
        L = X_train*W.T 
        L = L + b*np.ones(L.shape) #bias
        O = np.apply_along_axis(exp_fun, 1, L)
        Wn = (1-2*alpha*regul)*W + alpha*(y_train - O)*X_train
        b = (1-alpha*regul)*b + np.sum(alpha*(y_train - O))
        W = Wn
    pbar.update()

W = W/45
b = b/45

# Testing
L_test = X_test * W.T 
L_test = L_test + b*np.ones(L_test.shape)
O_test = np.apply_along_axis(exp_fun, 1, L_test)
acc = 1-(np.sum(np.around(np.absolute(y_test - O_test)))/5000)
print('Eficiencia: %.3f' % acc)

0%                          100%


Eficiencia: 0.891


In [25]:
import pyprind
doc_stream = stream_docs(path='shuffled_movie_data.csv')
X_in, y_in = get_minibatch(doc_stream, size=45000)

pbar = pyprind.ProgBar(45)
clf = SGDClassifier(loss='log', random_state=1, max_iter=1)
classes = np.array([0, 1])
for i in range(45):
    X_train = X_in[1000*i:1000*(i+1)]
    y_train = y_in[1000*i:1000*(i+1)]
    X_train = vect.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    pbar.update()

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:01:43


Depending on your machine, it will take about 2-3 minutes to stream the documents and learn the weights for the logistic regression model to classify "new" movie reviews. Executing the preceding code, we used the first 45,000 movie reviews to train the classifier, which means that we have 5,000 reviews left for testing:

In [ ]:
print('Accuracy: %.3f' % clf.score(X_test, y_test))

I think that the predictive performance, an accuracy of ~87%, is quite "reasonable" given that we "only" used the default parameters and didn't do any hyperparameter optimization. 

After we estimated the model perfomance, let us use those last 5,000 test samples to update our model.

In [ ]:
clf = clf.partial_fit(X_test, y_test)

<br>
<br>

# Model Persistence

In the previous section, we successfully trained a model to predict the sentiment of a movie review. Unfortunately, if we'd close this IPython notebook at this point, we'd have to go through the whole learning process again and again if we'd want to make a prediction on "new data."

So, to reuse this model, we could use the [`pickle`](https://docs.python.org/3.5/library/pickle.html) module to "serialize a Python object structure". Or even better, we could use the [`joblib`](https://pypi.python.org/pypi/joblib) library, which handles large NumPy arrays more efficiently.

To install:
conda install -c anaconda joblib

In [ ]:
import joblib
import os
if not os.path.exists('./pkl_objects'):
    os.mkdir('./pkl_objects')
    
joblib.dump(vect, './vectorizer.pkl')
joblib.dump(clf, './clf.pkl')

Using the code above, we "pickled" the `HashingVectorizer` and the `SGDClassifier` so that we can re-use those objects later. However, `pickle` and `joblib` have a known issue with `pickling` objects or functions from a `__main__` block and we'd get an `AttributeError: Can't get attribute [x] on <module '__main__'>` if we'd unpickle it later. Thus, to pickle the `tokenizer` function, we can write it to a file and import it to get the `namespace` "right".

In [ ]:
%%writefile tokenizer.py
from nltk.stem.porter import PorterStemmer
import re
from nltk.corpus import stopwords

stop = stopwords.words('english')
porter = PorterStemmer()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) + ' '.join(emoticons).replace('-', '')
    text = [w for w in text.split() if w not in stop]
    tokenized = [porter.stem(w) for w in text]
    return text

In [ ]:
from tokenizer import tokenizer
joblib.dump(tokenizer, './tokenizer.pkl')

Now, let us restart this IPython notebook and check if the we can load our serialized objects:

In [ ]:
import joblib
tokenizer = joblib.load('./tokenizer.pkl')
vect = joblib.load('./vectorizer.pkl')
clf = joblib.load('./clf.pkl')

After loading the `tokenizer`, `HashingVectorizer`, and the tranined logistic regression model, we can use it to make predictions on new data, which can be useful, for example, if we'd want to embed our classifier into a web application -- a topic for another IPython notebook.

In [ ]:
example = ['I did not like this movie']
X = vect.transform(example)
clf.predict(X)

In [ ]:
example = ['I loved this movie']
X = vect.transform(example)
clf.predict(X)